In [61]:
import pandas as pd
from Bio import SeqIO
import re
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
ncbi_aln = '../Avail_seqs/aligned/SNV_S_NCBI_MEZAP.aligned.fasta'
ncbi_data = '../Avail_seqs/ncbi_data.tsv'


In [12]:
#Load metadata
ncbi_df = pd.read_csv(ncbi_data, sep='\t')

#load aln
aln = SeqIO.to_dict(SeqIO.parse(ncbi_aln, 'fasta'))

#Function to split the header for NCBI

def split_ncbi_header(input_dict, separator):
    new_dict = {}
    for key, value in input_dict.items():
        if isinstance(key, str):
            split_keys = key.split(separator)
            for k in split_keys:
                stripped_key = k.strip()
                if not stripped_key:
                    continue
                new_dict[stripped_key] = value
        else:
            new_dict[key] = value
    return new_dict


#remove version numbers
aln_nover = split_ncbi_header(aln, '.')

dict_values([SeqRecord(seq=Seq('----------------------------------actccttgagaagctactac...---'), id='2DZNTT_1_1_SNVT_MIPE_222', name='2DZNTT_1_1_SNVT_MIPE_222', description='2DZNTT_1_1_SNVT_MIPE_222', dbxrefs=[]), SeqRecord(seq=Seq('----------------------------------actccttgagaagctactac...---'), id='YMD9RR_4_SNVT_PEMA_295', name='YMD9RR_4_SNVT_PEMA_295', description='YMD9RR_4_SNVT_PEMA_295', dbxrefs=[]), SeqRecord(seq=Seq('----------------------------------actccttgagaagctactac...---'), id='2DZNTT_2_2_SNVT_PEMA_165', name='2DZNTT_2_2_SNVT_PEMA_165', description='2DZNTT_2_2_SNVT_PEMA_165', dbxrefs=[]), SeqRecord(seq=Seq('----------------------------------actccttgagaagctactac...---'), id='2DZNTT_5_5_SNVT_PEMA_229', name='2DZNTT_5_5_SNVT_PEMA_229', description='2DZNTT_5_5_SNVT_PEMA_229', dbxrefs=[]), SeqRecord(seq=Seq('----------------------------------actccttgagaagctactac...---'), id='2DZNTT_6_6_SNVT_PEMA_230', name='2DZNTT_6_6_SNVT_PEMA_230', description='2DZNTT_6_6_SNVT_PEMA_230', dbxref

In [25]:
# Extract accessions and sequences excluding SNVT entries
accessions_ncbi = [
    key for key in aln_nover.keys()
    if "SNVT" not in key
]

sequences_ncbi = [
    str(aln_nover[key].seq) for key in aln_nover.keys()
    if "SNVT" not in key
]

# Create DataFrame of aligned sequences
ncbi_df_aln = pd.DataFrame({
    'accession': accessions_ncbi,
    'sequence': sequences_ncbi
})

# Standardize column names
ncbi_df.columns = ncbi_df.columns.str.lower()

# Merge sequence data with metadata
ncbi_df_merged = pd.merge(ncbi_df, ncbi_df_aln, on='accession', how='inner')

# Assign two-letter state codes based on submitter name
def add_state_code(last_name, data, state_code):
    for index, row in data.iterrows():
        if last_name in row['submitters']:
            data.at[index, 'usa'] = state_code
    return data

# Apply mapping of submitter names to state codes
names_state_dict = {
    'Goodfellow': 'NM',
    'Hecht': 'AZ',
    'Hjelle': 'NM',
    'Botten': 'NM',
    'Spiropoulou': 'NM'
}

ncbi_df_merged_test = ncbi_df_merged.copy()

for last_name, state_code in names_state_dict.items():
    ncbi_df_merged_test = add_state_code(last_name, ncbi_df_merged_test, state_code)


#ncbi_df_merged_test.to_csv('../Avail_seqs/ncbi_data_aln.tsv', sep='\t', index=False) #Continued manually

In [54]:
#load whitman metadata
whitman2023 = pd.read_excel('../Avail_seqs/rodent_sero_ELISAdata.xlsx', sheet_name='RNAextractionLung')
whitman2025 = pd.read_excel('../Avail_seqs/WHIT.Samples.EEIDP.xlsx', sheet_name='3.3.25 - RNA Extraction')

whitman2023 = whitman2023[['Individual', 'Date Sampled']]
whitman2023.columns = ['sample', 'collection_date']
whitman2025 = whitman2025[['Sample ID Number', 'Collection Date']]
whitman2025.columns = ['sample', 'collection_date']

#concatenate
whitman_samples = pd.concat([whitman2023, whitman2025], ignore_index=True)
#Format dates as yyyy-mm-dd
whitman_samples['collection_date'] = pd.to_datetime(whitman_samples['collection_date'], errors='coerce').dt.strftime('%Y-%m-%d')
#drop samples without dates
whitman_samples = whitman_samples.dropna(subset=['collection_date'])

#Dataframe for whitman sequences
accessions_whitman = [
    key for key in aln_nover.keys()
    if "SNVT" in key
]

#Change PEMA for PESO
for key in accessions_whitman:
    if "PEMA" in key:
        new_key = key.replace("PEMA", "PESO")
        aln_nover[new_key] = aln_nover[key]

sequences_whitman = [
    str(aln_nover[key].seq) for key in aln_nover.keys()
    if "SNVT" in key
]

# Create DataFrame of aligned sequences
whitman_df_aln = pd.DataFrame({
    'accession': accessions_whitman,
    'sequence': sequences_whitman,
    'usa' : 'WA'
})

def extract_sample_name(accession):
    if "WHIT" in accession:
        match = re.search(r'(WHIT)0?(\d{1,2})', accession)
        if match:
            return match.group(1) + match.group(2)
    if "EP" in accession:
        match = re.search(r'(EP)0?(\d*)', accession)
        if match:
            return match.group(2)
    else:
        return accession.split('_')[-1]
    
whitman_df_aln['sample'] = whitman_df_aln['accession'].apply(extract_sample_name)

#extract sample names for the metadata
whitman_samples['sample'] = whitman_samples['sample'].apply(extract_sample_name)

#add the collection date to the whitman_df_aln
whitman_df_aln = pd.merge(whitman_df_aln, whitman_samples, on='sample', how='inner')
whitman_df_aln


,accession,sequence,usa,sample,collection_date
0,2DZNTT_1_1_SNVT_MIPE_222,----------------------------------actccttgagaa...,WA,222,2023-06-28
1,YMD9RR_1_SNVT_MIPE_223,----------------------------------actccttgagaa...,WA,223,2023-06-28
2,YMD9RR_4_SNVT_PESO_295,----------------------------------actccttgagaa...,WA,295,2023-07-17
3,2DZNTT_2_2_SNVT_PESO_165,----------------------------------actccttgagaa...,WA,165,2023-06-12
4,2DZNTT_5_5_SNVT_PESO_229,----------------------------------actccttgagaa...,WA,229,2023-06-28
5,2DZNTT_6_6_SNVT_PESO_230,----------------------------------actccttgagaa...,WA,230,2023-06-28
6,2DZNTT_4_4_SNVT_PESO_196,----------------------------------actccttgagaa...,WA,196,2023-06-22
7,YMD9RR_2_SNVT_PESO_261,----------------------------------actccttgagaa...,WA,261,2023-07-12
8,YMD9RR_3_SNVT_PESO_287,----------------------------------actccttgagaa...,WA,287,2023-07-17
9,2DZNTT_3_3_SNVT_PESO_180,---------------------------------gactccttgagaa...,WA,180,2023-06-18


In [63]:
#Load the NCBI metadata (again)

filled_ncbi = pd.read_csv('../Avail_seqs/ncbi_data_aln.tsv', sep='\t')

filled_ncbi.columns

#Select columns to match the whitman columns
filled_ncbi = filled_ncbi[['accession', 'sequence','usa', 'collection_date']]
filled_ncbi['sample'] = filled_ncbi['accession']

#concatenate whitman and ncbi
ncbi_whitman_df = pd.concat([filled_ncbi, whitman_df_aln], ignore_index=True)

#fix dates

def normalize_collection_date(date_str):
    if pd.isna(date_str):
        return date_str 
    # Remove non-numeric characters and count digits
    num_digits = sum(c.isdigit() for c in date_str)

    if num_digits == 4:
        return date_str + "-06-01"  # Only year
    elif num_digits == 6:
        return date_str + "-01"     # Year + month
    elif num_digits == 8:
        return date_str             # Full date
    else:
        return pd.NaT  # Invalid or malformed date
    

#apply to the function
ncbi_whitman_df['collection_date'] = ncbi_whitman_df['collection_date'].apply(normalize_collection_date)

#create new header
ncbi_whitman_df['header'] = ncbi_whitman_df['sample'] + '|' + ncbi_whitman_df['usa'] + '|' + ncbi_whitman_df['collection_date'].astype(str)

#Write aligned sequences to a new file
records = [
    SeqRecord(Seq(seq), id=sample, description="")
    for sample, seq in zip(ncbi_whitman_df['header'], ncbi_whitman_df['sequence'])
]

with open('../Avail_seqs/ncbi_whitman_aligned.fasta', 'w') as output_handle:
    SeqIO.write(records, output_handle, 'fasta')
